<a href="https://colab.research.google.com/github/pushkar-khetrapal/EfficientPS/blob/master/EfficientPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install efficientnet_pytorch
!pip install git+https://github.com/mapillary/inplace_abn

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=477c5e925ab560851accd402e3eba79756803522582b98977be4cf8178181907
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch
  Cloning https://github.com/mapillary/inplace_abn to /tmp/pip-req-build-adf51yab
  Running command git clone -q https://github.com/mapillary/inplace_abn /tmp/pip-req-build-adf51yab
  Created wheel for inplace-abn: filename=inplace_abn-1.0.12-cp36-cp36m-linux_x86_64.whl size=3264503 sha256=d032e4035ca4cff8b5bf97f5835a154072dffc8f96561bb921a983f42f446164
  Stored in directory: /tmp/pip-ephem-wheel-cache-1lwjc2mn/wheels/fe/0b/49/1303ca37166cc1be8784e2367a172133634dcd864a9df0ab56
Successfully built inplace-abn


In [2]:
from efficientnet_pytorch import utils

In [2]:
import collections
import re
# Parameters for the entire model (stem, all blocks, and head)
GlobalParams = collections.namedtuple('GlobalParams', [
    'width_coefficient', 'depth_coefficient', 'image_size', 'dropout_rate',
    'num_classes', 'batch_norm_momentum', 'batch_norm_epsilon',
    'drop_connect_rate', 'depth_divisor', 'min_depth'])

# Parameters for an individual model block
BlockArgs = collections.namedtuple('BlockArgs', [
    'num_repeat', 'kernel_size', 'stride', 'expand_ratio',
    'input_filters', 'output_filters', 'se_ratio', 'id_skip'])

In [3]:
def efficientnet_params(model_name):
    """ Map EfficientNet model name to parameter coefficients. """
    params_dict = {
        # Coefficients:   width,depth,res,dropout
        'efficientnet-b0': (1.0, 1.0, 224, 0.2),
        'efficientnet-b1': (1.0, 1.1, 240, 0.2),
        'efficientnet-b2': (1.1, 1.2, 260, 0.3),
        'efficientnet-b3': (1.2, 1.4, 300, 0.3),
        'efficientnet-b4': (1.4, 1.8, 380, 0.4),
        'efficientnet-b5': (1.6, 2.2, 456, 0.4),
        'efficientnet-b6': (1.8, 2.6, 528, 0.5),
        'efficientnet-b7': (2.0, 3.1, 600, 0.5),
        'efficientnet-b8': (2.2, 3.6, 672, 0.5),
        'efficientnet-l2': (4.3, 5.3, 800, 0.5),
    }
    return params_dict[model_name]

class BlockDecoder(object):
    """ Block Decoder for readability, straight from the official TensorFlow repository """

    @staticmethod
    def _decode_block_string(block_string):
        """ Gets a block through a string notation of arguments. """
        assert isinstance(block_string, str)

        ops = block_string.split('_')
        options = {}
        for op in ops:
            splits = re.split(r'(\d.*)', op)
            if len(splits) >= 2:
                key, value = splits[:2]
                options[key] = value

        # Check stride
        assert (('s' in options and len(options['s']) == 1) or
                (len(options['s']) == 2 and options['s'][0] == options['s'][1]))

        return BlockArgs(
            kernel_size=int(options['k']),
            num_repeat=int(options['r']),
            input_filters=int(options['i']),
            output_filters=int(options['o']),
            expand_ratio=int(options['e']),
            id_skip=('noskip' not in block_string),
            se_ratio=float(options['se']) if 'se' in options else None,
            stride=[int(options['s'][0])])

    @staticmethod
    def _encode_block_string(block):
        """Encodes a block to a string."""
        args = [
            'r%d' % block.num_repeat,
            'k%d' % block.kernel_size,
            's%d%d' % (block.strides[0], block.strides[1]),
            'e%s' % block.expand_ratio,
            'i%d' % block.input_filters,
            'o%d' % block.output_filters
        ]
        if 0 < block.se_ratio <= 1:
            args.append('se%s' % block.se_ratio)
        if block.id_skip is False:
            args.append('noskip')
        return '_'.join(args)

    @staticmethod
    def decode(string_list):
        """
        Decodes a list of string notations to specify blocks inside the network.

        :param string_list: a list of strings, each string is a notation of block
        :return: a list of BlockArgs namedtuples of block args
        """
        assert isinstance(string_list, list)
        blocks_args = []
        for block_string in string_list:
            blocks_args.append(BlockDecoder._decode_block_string(block_string))
        return blocks_args

    @staticmethod
    def encode(blocks_args):
        """
        Encodes a list of BlockArgs to a list of strings.

        :param blocks_args: a list of BlockArgs namedtuples of block args
        :return: a list of strings, each string is a notation of block
        """
        block_strings = []
        for block in blocks_args:
            block_strings.append(BlockDecoder._encode_block_string(block))
        return block_strings


def efficientnet(width_coefficient=None, depth_coefficient=None, dropout_rate=0.2,
                 drop_connect_rate=0.2, image_size=None, num_classes=1000):
    """ Creates a efficientnet model. """

    blocks_args = [
        'r1_k3_s11_e1_i32_o16_se0.25', 'r2_k3_s22_e6_i16_o24_se0.25',
        'r2_k5_s22_e6_i24_o40_se0.25', 'r3_k3_s22_e6_i40_o80_se0.25',
        'r3_k5_s11_e6_i80_o112_se0.25', 'r4_k5_s22_e6_i112_o192_se0.25',
        'r1_k3_s11_e6_i192_o320_se0.25',
    ]
    blocks_args = BlockDecoder.decode(blocks_args)

    global_params = GlobalParams(
        batch_norm_momentum=0.99,
        batch_norm_epsilon=1e-3,
        dropout_rate=dropout_rate,
        drop_connect_rate=drop_connect_rate,
        # data_format='channels_last',  # removed, this is always true in PyTorch
        num_classes=num_classes,
        width_coefficient=width_coefficient,
        depth_coefficient=depth_coefficient,
        depth_divisor=8,
        min_depth=None,
        image_size=image_size,
    )

    return blocks_args, global_params


def get_model_params(model_name, override_params):
    """ Get the block args and global params for a given model """
    if model_name.startswith('efficientnet'):
        w, d, s, p = efficientnet_params(model_name)
        # note: all models have drop connect rate = 0.2
        blocks_args, global_params = efficientnet(
            width_coefficient=w, depth_coefficient=d, dropout_rate=p, image_size=s)
    else:
        raise NotImplementedError('model name is not pre-defined: %s' % model_name)
    if override_params:
        # ValueError will be raised here if override_params has fields not included in global_params.
        global_params = global_params._replace(**override_params)
    return blocks_args, global_params

In [4]:
override_params={'num_classes': 1000}
x = get_model_params( 'efficientnet-b5', override_params )

In [5]:
x[0]

[BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=1, input_filters=32, output_filters=16, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=2, kernel_size=3, stride=[2], expand_ratio=6, input_filters=16, output_filters=24, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=2, kernel_size=5, stride=[2], expand_ratio=6, input_filters=24, output_filters=40, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=3, kernel_size=3, stride=[2], expand_ratio=6, input_filters=40, output_filters=80, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=3, kernel_size=5, stride=[1], expand_ratio=6, input_filters=80, output_filters=112, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=4, kernel_size=5, stride=[2], expand_ratio=6, input_filters=112, output_filters=192, se_ratio=0.25, id_skip=True),
 BlockArgs(num_repeat=1, kernel_size=3, stride=[1], expand_ratio=6, input_filters=192, output_filters=320, se_ratio=0.25, id_skip=True)]

In [6]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet(x[0],x[1] )

In [7]:
import torch

In [8]:
!pip install torch-summary

In [9]:
import torch
from torchsummary import summary

summary(model, torch.zeros((1, 3, 1024, 2048)))
#print(summary(model, torch.zeros((1, 3, 1024, 2048)), show_input=True, show_hierarchical=True))

----------------------------------------------------------------------------------------------------
Layer (type:depth-idx)                             Output Shape              Param #
├─Conv2dStaticSamePadding: 1-1                     [-1, 48, 512, 1024]       --
|    └─ZeroPad2d: 2-1                              [-1, 3, 1025, 2049]       --
├─BatchNorm2d: 1-2                                 [-1, 48, 512, 1024]       96
├─MemoryEfficientSwish: 1-3                        [-1, 48, 512, 1024]       --
├─Conv2dStaticSamePadding: 1-4                     [-1, 2048, 32, 64]        --
|    └─MBConvBlock: 2-2                            [-1, 24, 512, 1024]       --
|    |    └─Conv2dStaticSamePadding: 3-1           [-1, 48, 512, 1024]       432
|    |    └─BatchNorm2d: 3-2                       [-1, 48, 512, 1024]       96
|    |    └─MemoryEfficientSwish: 3-3              [-1, 48, 512, 1024]       --
|    |    └─Conv2dStaticSamePadding: 3-4           [-1, 12, 1, 1]            588
|    |    └─

----------------------------------------------------------------------------------------------------
Layer (type:depth-idx)                             Output Shape              Param #
├─Conv2dStaticSamePadding: 1-1                     [-1, 48, 512, 1024]       --
|    └─ZeroPad2d: 2-1                              [-1, 3, 1025, 2049]       --
├─BatchNorm2d: 1-2                                 [-1, 48, 512, 1024]       96
├─MemoryEfficientSwish: 1-3                        [-1, 48, 512, 1024]       --
├─Conv2dStaticSamePadding: 1-4                     [-1, 2048, 32, 64]        --
|    └─MBConvBlock: 2-2                            [-1, 24, 512, 1024]       --
|    |    └─Conv2dStaticSamePadding: 3-1           [-1, 48, 512, 1024]       432
|    |    └─BatchNorm2d: 3-2                       [-1, 48, 512, 1024]       96
|    |    └─MemoryEfficientSwish: 3-3              [-1, 48, 512, 1024]       --
|    |    └─Conv2dStaticSamePadding: 3-4           [-1, 12, 1, 1]            588
|    |    └─

In [14]:
import torch
from torch import nn
from torch.nn import functional as F
from inplace_abn.abn import InPlaceABN

from efficientnet_pytorch.utils import (
    round_filters,
    round_repeats,
    drop_connect,
    get_same_padding_conv2d,
    get_model_params,
    efficientnet_params,
    load_pretrained_weights,
    Swish,
    MemoryEfficientSwish,
)

class MBConvBlock(nn.Module):
    """
    Mobile Inverted Residual Bottleneck Block

    Args:
        block_args (namedtuple): BlockArgs, see above
        global_params (namedtuple): GlobalParam, see above

    Attributes:
        has_se (bool): Whether the block contains a Squeeze and Excitation layer.
    """

    def __init__(self, block_args, global_params):
        super().__init__()
        self._block_args = block_args
        self._bn_mom = 1 - global_params.batch_norm_momentum
        self._bn_eps = global_params.batch_norm_epsilon
        self.has_se = (self._block_args.se_ratio is not None) and (0 < self._block_args.se_ratio <= 1)
        self.id_skip = block_args.id_skip  # skip connection and drop connect

        # Get static or dynamic convolution depending on image size
        Conv2d = get_same_padding_conv2d(image_size=global_params.image_size)

        # Expansion phase
        inp = self._block_args.input_filters  # number of input channels
        oup = self._block_args.input_filters * self._block_args.expand_ratio  # number of output channels
        if self._block_args.expand_ratio != 1:
            self._expand_conv = Conv2d(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
            self._bn0 = InPlaceABN(oup)

        # Depthwise convolution phase
        k = self._block_args.kernel_size
        s = self._block_args.stride
        self._depthwise_conv = Conv2d(
            in_channels=oup, out_channels=oup, groups=oup,  # groups makes it depthwise
            kernel_size=k, stride=s, bias=False)
        self._bn1 = InPlaceABN(oup)

        # Squeeze and Excitation layer, if desired

        ## Deleted it from here


        # Output phase
        final_oup = self._block_args.output_filters
        self._project_conv = Conv2d(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
        self._bn2 = InPlaceABN(final_oup)

    def forward(self, inputs, drop_connect_rate=None):
        """
        :param inputs: input tensor
        :param drop_connect_rate: drop connect rate (float, between 0 and 1)
        :return: output of block
        """

        # Expansion and Depthwise Convolution
        x = inputs
        if self._block_args.expand_ratio != 1:
            x = self._bn0(self._expand_conv(inputs))
        x = self._bn1(self._depthwise_conv(x))

        # Squeeze and Excitation
        # Delete SE layer from here


        x = self._bn2(self._project_conv(x))

        # Skip connection and drop connect
        input_filters, output_filters = self._block_args.input_filters, self._block_args.output_filters
        if self.id_skip and self._block_args.stride == 1 and input_filters == output_filters:
            if drop_connect_rate:
                x = drop_connect(x, p=drop_connect_rate, training=self.training)
            x = x + inputs  # skip connection
        return x




class EfficientNet(nn.Module):
    """
    An EfficientNet model. Most easily loaded with the .from_name or .from_pretrained methods

    Args:
        blocks_args (list): A list of BlockArgs to construct blocks
        global_params (namedtuple): A set of GlobalParams shared between blocks

    Example:
        model = EfficientNet.from_pretrained('efficientnet-b0')

    """

    def __init__(self, blocks_args=None, global_params=None):
        super().__init__()
        assert isinstance(blocks_args, list), 'blocks_args should be a list'
        assert len(blocks_args) > 0, 'block args must be greater than 0'
        self._global_params = global_params
        self._blocks_args = blocks_args

        # Get static or dynamic convolution depending on image size
        Conv2d = get_same_padding_conv2d(image_size=global_params.image_size)

        # Batch norm parameters
        bn_mom = 1 - self._global_params.batch_norm_momentum
        bn_eps = self._global_params.batch_norm_epsilon

        # Stem
        in_channels = 3  # rgb
        out_channels = round_filters(32, self._global_params)  # number of output channels
        self._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        self._bn0 = InPlaceABN(out_channels)

        # Build blocks
        self._blocks = nn.ModuleList([])

        for block_args in self._blocks_args:

            # Update block input and output filters based on depth multiplier.
            block_args = block_args._replace(
                input_filters=round_filters(block_args.input_filters, self._global_params),
                output_filters=round_filters(block_args.output_filters, self._global_params),
                num_repeat=round_repeats(block_args.num_repeat, self._global_params)
            )

            # The first block needs to take care of stride and filter size increase.
            self._blocks.append(MBConvBlock(block_args, self._global_params))
            if block_args.num_repeat > 1:
                block_args = block_args._replace(input_filters=block_args.output_filters, stride=1)
            for _ in range(block_args.num_repeat - 1):
                self._blocks.append(MBConvBlock(block_args, self._global_params))


        # Head
        in_channels = block_args.output_filters  # output of final block
        out_channels = round_filters(1280, self._global_params)
        self._conv_head = Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self._bn1 = InPlaceABN(out_channels)


    def extract_features(self, inputs):
        """ Returns output of the final convolution layer """

        # Stem
        x = self._bn0(self._conv_stem(inputs))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)
            x = block(x, drop_connect_rate=drop_connect_rate)
            


        # Head
        x = self._bn1(self._conv_head(x))
        #self.arr.append(x)
        return x

    def forward(self, inputs):
        """ Calls extract_features to extract features, applies final linear layer, and returns logits. """
        bs = inputs.size(0)
        # Convolution layers
        x = self.extract_features(inputs)

        return x

    @classmethod
    def from_name(cls, model_name, override_params=None):
        cls._check_model_name_is_valid(model_name)
        blocks_args, global_params = get_model_params(model_name, override_params)
        return cls(blocks_args, global_params)

    @classmethod
    def from_pretrained(cls, model_name, advprop=False, num_classes=1000, in_channels=3):
        model = cls.from_name(model_name, override_params={'num_classes': num_classes})
        load_pretrained_weights(model, model_name, load_fc=(num_classes == 1000), advprop=advprop)
        if in_channels != 3:
            Conv2d = get_same_padding_conv2d(image_size = model._global_params.image_size)
            out_channels = round_filters(32, model._global_params)
            model._conv_stem = Conv2d(in_channels, out_channels, kernel_size=3, stride=2, bias=False)
        return model
    
    @classmethod
    def get_image_size(cls, model_name):
        cls._check_model_name_is_valid(model_name)
        _, _, res, _ = efficientnet_params(model_name)
        return res

    @classmethod
    def _check_model_name_is_valid(cls, model_name):
        """ Validates model name. """ 
        valid_models = ['efficientnet-b'+str(i) for i in range(9)]
        if model_name not in valid_models:
            raise ValueError('model_name should be one of: ' + ', '.join(valid_models))

    def return_sub(self):
      return self._blocks


In [15]:
model = EfficientNet(x[0],x[1])

In [12]:
arr = model.return_sub()

In [16]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): InPlaceABN(48, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): InPlaceABN(48, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): InPlaceABN(24, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
    )
    (1): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        24, 24, kernel_size=(3, 3), stride=(1, 1), grou

In [22]:
arr

[MBConvBlock(
   (_depthwise_conv): Conv2dStaticSamePadding(
     24, 24, kernel_size=(3, 3), stride=(1, 1), groups=24, bias=False
     (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
   )
   (_bn1): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_project_conv): Conv2dStaticSamePadding(
     24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
     (static_padding): Identity()
   )
   (_bn2): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_swish): MemoryEfficientSwish()
 ), MBConvBlock(
   (_depthwise_conv): Conv2dStaticSamePadding(
     24, 24, kernel_size=(3, 3), stride=(1, 1), groups=24, bias=False
     (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
   )
   (_bn1): BatchNorm2d(24, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
   (_project_conv): Conv2dStaticSamePadding(
     24, 24, kernel_size=(1, 1), stride=(1, 1

In [17]:
import torch
from torchsummary import summary

sum = summary(model, torch.zeros((1, 3, 1024, 2048)))
#print(summary(model, torch.zeros((1, 3, 1024, 2048)), show_input=True, show_hierarchical=True))

----------------------------------------------------------------------------------------------------
Layer (type:depth-idx)                             Output Shape              Param #
├─Conv2dStaticSamePadding: 1-1                     [-1, 48, 512, 1024]       --
|    └─ZeroPad2d: 2-1                              [-1, 3, 1025, 2049]       --
├─InPlaceABN: 1-2                                  [-1, 48, 512, 1024]       96
├─Conv2dStaticSamePadding: 1-3                     [-1, 2048, 32, 64]        --
|    └─MBConvBlock: 2-2                            [-1, 24, 512, 1024]       --
|    |    └─Conv2dStaticSamePadding: 3-1           [-1, 48, 512, 1024]       432
|    |    └─InPlaceABN: 3-2                        [-1, 48, 512, 1024]       96
|    |    └─Conv2dStaticSamePadding: 3-3           [-1, 24, 512, 1024]       1,152
|    |    └─InPlaceABN: 3-4                        [-1, 24, 512, 1024]       48
|    └─MBConvBlock: 2-3                            [-1, 24, 512, 1024]       --
|    |    

In [23]:
out = model(torch.randn((1, 3, 1024, 2048)).to('cuda'))

In [25]:
out

tensor([[[[ 1.7473e-01, -2.3720e-06, -8.6524e-03,  ..., -9.0770e-03,
            7.1543e-01,  1.3273e+00],
          [ 1.7386e+00,  3.1725e-01,  2.7964e-01,  ...,  1.0822e+00,
           -8.7940e-05, -8.5076e-04],
          [ 5.6941e-01,  2.1081e-01, -6.9113e-03,  ..., -9.4085e-03,
           -1.2389e-02,  3.0728e+00],
          ...,
          [-1.2073e-04,  4.3526e-01,  1.1212e+00,  ..., -6.4945e-03,
            2.0077e-01,  3.5688e-01],
          [ 5.2350e-01, -5.2866e-03, -1.3075e-02,  ..., -7.1414e-03,
            7.6836e-01,  8.4422e-03],
          [ 1.7307e+00,  1.9177e-01,  1.3171e+00,  ..., -2.1677e-03,
            1.4745e+00,  6.1034e-01]],

         [[-9.8656e-03,  1.0582e+00,  8.8826e-01,  ..., -1.0193e-02,
            6.3557e-01,  9.7449e-01],
          [-1.6288e-02, -2.0732e-03,  1.1458e+00,  ...,  5.2858e-01,
           -1.4589e-02, -3.9106e-03],
          [ 2.8597e-01,  6.8582e-01,  3.0464e-01,  ...,  1.4267e+00,
            8.7797e-01,  9.9169e-01],
          ...,
     

In [18]:
model._blocks

ModuleList(
  (0): MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): InPlaceABN(48, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
    (_project_conv): Conv2dStaticSamePadding(
      48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )
    (_bn2): InPlaceABN(24, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
  )
  (1): MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      24, 24, kernel_size=(3, 3), stride=(1, 1), groups=24, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): InPlaceABN(24, eps=1e-05, momentum=0.1, affine=True, activation=leaky_relu[0.01])
    (_project_conv): Conv2dStaticSamePadding(
      24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): I